In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
header= ["레시피일련번호",'레시피제목','요리명','등록자ID','등록자명','조회수','추천수','스크랩수','요리방법별명','요리상황별명','요리재료별명','요리종류별명','요리소개','요리재료내용','요리인분명','요리난이도명',"요리시간명","최초등록일시"]
ckg = pd.read_csv('../../data/만개의레시피.csv', header=None, names=header, encoding='ansi')
ckg.drop(0, inplace=True)
ckg.info()

C:\Users\user\AppData\Local\Temp\ipykernel_10588\2068350340.py:2: DtypeWarning: Columns (0,5,6,7,17) have mixed types. Specify dtype option on import or set low_memory=False.
  ckg = pd.read_csv('../../data/만개의레시피.csv', header=None, names=header, encoding='ansi')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128400 entries, 1 to 128400
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   레시피일련번호  128400 non-null  object
 1   레시피제목    128400 non-null  object
 2   요리명      126310 non-null  object
 3   등록자ID    128400 non-null  object
 4   등록자명     128394 non-null  object
 5   조회수      128400 non-null  object
 6   추천수      128400 non-null  object
 7   스크랩수     128400 non-null  object
 8   요리방법별명   128400 non-null  object
 9   요리상황별명   127510 non-null  object
 10  요리재료별명   128400 non-null  object
 11  요리종류별명   128400 non-null  object
 12  요리소개     127783 non-null  object
 13  요리재료내용   127340 non-null  object
 14  요리인분명    124171 non-null  object
 15  요리난이도명   128137 non-null  object
 16  요리시간명    120054 non-null  object
 17  최초등록일시   128400 non-null  object
dtypes: object(18)
memory usage: 17.6+ MB


In [3]:
ckg.drop(['등록자ID','등록자명','요리인분명','요리난이도명',"요리시간명","최초등록일시"], axis=1, inplace=True)
ckg.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용
1,128671,어묵김말이,어묵김말이,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...
2,128892,두부에 꼬리가 달렸어요!!,NaN,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...
3,128932,입안에서 톡톡톡,NaN,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...
4,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
5,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...


In [4]:
#요리재료내용 결측치 삭제
ckg_drop = ckg[ckg.loc[:,'요리재료내용'].isnull()].index
ckg_drop = ckg.drop(ckg_drop)
ckg_drop.reset_index(drop=True, inplace=True)
ckg_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127340 entries, 0 to 127339
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   레시피일련번호  127340 non-null  object
 1   레시피제목    127340 non-null  object
 2   요리명      125389 non-null  object
 3   조회수      127340 non-null  object
 4   추천수      127340 non-null  object
 5   스크랩수     127340 non-null  object
 6   요리방법별명   127340 non-null  object
 7   요리상황별명   126451 non-null  object
 8   요리재료별명   127340 non-null  object
 9   요리종류별명   127340 non-null  object
 10  요리소개     126859 non-null  object
 11  요리재료내용   127340 non-null  object
dtypes: object(12)
memory usage: 11.7+ MB


In [16]:
ckg_drop.요리종류별명.value_counts()

밑반찬         25346
메인반찬        20343
밥/죽/떡       17447
빵           11834
국/탕         10404
면/만두         8585
디저트          6457
찌개           4092
김치/젓갈/장류     3855
기타           3690
과자           3393
차/음료/술       2863
양식           2796
샐러드          2746
양념/소스/잼      2099
퓨전            820
스프            570
Name: 요리종류별명, dtype: int64

In [17]:
ckg_drop.요리재료별명.value_counts()

채소류       33865
해물류       14102
밀가루       12692
가공식품류     12136
돼지고기       7698
달걀/유제품     7540
소고기        5763
쌀          5597
닭고기        5372
기타         5263
과일류        4328
콩/견과류      4205
건어물류       3735
버섯류        1713
육류         1712
곡류         1619
Name: 요리재료별명, dtype: int64

In [5]:
ckg_drop.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용
0,128671,어묵김말이,어묵김말이,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...
1,128892,두부에 꼬리가 달렸어요!!,NaN,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...
2,128932,입안에서 톡톡톡,NaN,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...
3,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...


In [22]:
#재료 전처리
ckg_drop['요리재료_전처리'] = np.NaN
#len(ckg_drop)
for idx in range(4, 8):
    ingredient = ckg_drop.loc[idx, '요리재료내용']
    regex = re.match('\[[^)]*\]', ingredient)
    print(regex)
    # ingredient = re.sub(r'\[재료\]|\[양념\]', ' ', str(ingredient)) #괄호안 문자, | 제거
    # ingredient = re.sub(r'\[[^)]*\]|\([^)]*\)|\|', ' ', str(ingredient)) 
    ingredient = ingredient.strip()
    ingredients = ingredient.split(' ')
    
    ingre_lists = []
    for i in ingredients:
        i_int = re.compile(r'[0-9]').search(i) #계량표기 제거
        if (i_int == None) and i_int != '':
            ingre_lists.append(i)
            
    ckg_drop.loc[idx, '요리재료_전처리'] = ' '.join(ingre_lists)

ckg_drop.head(8)

<re.Match object; span=(0, 26), match='[재료] 북어포 1마리| 찹쌀가루 1C [양념]'>
<re.Match object; span=(0, 14), match='[재료] 파스타면 [양념]'>
<re.Match object; span=(0, 100), match='[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1/2쪽>
<re.Match object; span=(0, 73), match='[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올리브유>


,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리
0,128671,어묵김말이,어묵김말이,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,NaN
1,128892,두부에 꼬리가 달렸어요!!,NaN,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,NaN
2,128932,입안에서 톡톡톡,NaN,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,NaN
3,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,NaN
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,[재료] 북어포 찹쌀가루 [양념] 간장 설탕 물 다진파 다진마늘 참기름 깨소금 후춧...
5,149207,토마토스파게티♥,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,[재료] 파스타면 [양념] 토마토 토마토 페이스트 양파 다진마늘 피망 올리브유
6,151148,표고버섯탕수,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,[재료] 건표고버섯 오이 당근 양파 사과 그외의 야채| 과일 [녹말물] 녹말가루 물...
7,153040,달콤한 마늘향이 가득해~ 갈릭치킨♥,갈릭치킨,8148,0,38,튀김,손님접대,닭고기,메인반찬,요즘 갈릭이 대세인거 같아요~ 그래서 오늘은 제가 너무 좋아하는 닭과 마늘을 한번에...,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...,[재료] 닭 [닭양념] 통마늘 다진마늘 화이트와인또는우유| 올리브유또는버터| 마늘솔...


In [56]:
#재료 전체 사전을 만들기 위한 작업
ingredient_series = ckg['요리재료내용'].dropna()
ingredient_list = []
for ingredient in ingredient_series:
    ingredient = re.sub(r'\[[^)]*\]|\([^)]*\)|\|', '', ingredient) #괄호안 문자, | 제거
    ingredient = ingredient.strip()
    ingredients = ingredient.split(' ')
    
    #계량표기 제거
    for i in ingredients:
        i_int = re.compile(r'[0-9]').search(i)
        if (i_int == None) and i_int != '':
            ingredient_list.append(i)

#중복값 제거
ingredient_list = list(set(ingredient_list))
ingredient_list

['',
 '페코리노치즈',
 '깍뚝썰은',
 '바늘',
 '청상추',
 '마무리간',
 '등신',
 '큐빅',
 '뼈삶은',
 '사과껍질',
 '가시오이',
 '비앤나소시지',
 '재량',
 '아가베시럽또는',
 '멍게무침',
 '무춘',
 '쿠킹오일',
 '겨자초절임무',
 '뉴수가or설탕',
 '실온계란',
 '양파갈은것',
 '코팅용초콜릿',
 '오분',
 '찜용콩나물',
 '황태다시마',
 '머핀]',
 'mixed',
 '[타르트틀',
 '사과용',
 '까놓은건',
 '핫케?',
 '바지락과홍합잠기게',
 '마카로니파스타',
 '카레분말가루',
 '류',
 '불린찹쌀',
 '[민트잎',
 '오올리고당',
 '?북어',
 '카놀라이유',
 '오이양파피클',
 '생강조금',
 '미리구운',
 '진달래',
 '세이지',
 '호막',
 '익스트렉',
 '가나초콜릿',
 '스테이크용씨즈닝',
 '도식락김',
 '쿡두',
 '양배추가루',
 '설탕a',
 '바지락조개살',
 '파랑천연색소',
 '바닐라',
 '천연육수이야기팩',
 '춘매',
 '작당히',
 '한봉지이',
 '곤젓',
 '설탕ⓐ',
 '흑설탕/황설탕',
 '고둥',
 '말린대추',
 '[오므라이스소스재료]',
 '가쓰오국물',
 '샐러드믹스',
 '&레몬즙',
 '김장',
 '마늘간거',
 '믹스과일',
 '스파게티용토마토소스',
 '김장비닐',
 '소내장수육',
 '딸기우유',
 '양배추&호박잎',
 '비트잎',
 '크래미',
 '몇조각',
 '냄새나는반찬통',
 '레시피마다',
 '반정도',
 '당',
 '멸치&다시마물',
 '적당것',
 '식용색소펜',
 '갠',
 '고무밴드',
 '가루들',
 '크리스마스장식',
 '약도라지',
 '젤리밥',
 '표고버섯밑둥',
 '팬웍에',
 '파프리카와동일한양',
 '나베육수',
 '해바라기씨등)',
 '우동쯔유',
 '소사골',
 '마리네이드',
 'Coconut',
 '먹을만치',
 '초코무스',
 '채끝살/부채살/등심/',
 '초코카스테라',
 

In [57]:
#텍스트 파일로 전체 재료 저장
with open('만개의레시피_재료_목록.txt', 'w') as f :
    for ingre in ingredient_list:
        f.write(ingre+',')

In [58]:
len(ingredient_list)

26672